In [ ]:
import pandas as pd

In [ ]:
df = pd.read_json("../data/dataset-go-critic-style.jsonl", lines=True)

labels = df.explode("labels")["labels"].value_counts()
print(labels)

In [ ]:
from datasets import load_dataset

ds = load_dataset("aholovko/go-critic-style", split="train")

In [ ]:
class_label = ds.features["labels"].feature

for label in class_label.names:
    print(f"{class_label.str2int(label)}\t{label}")